In [2]:
def perpixel_filter_direct(blue, green, red, nir, swir1, swir2, tsmask):

    """
    Function Name: perpixel_filter_direct
    Description: 
    
    This function performs time series cloud/shadow detection for one pixel
  
    Parameters: 
    
    blue, green, red, nir, swir1, swir2: float, 1D arrays
        Surface reflectance time series data of band blue, green, red, nir, swir1, swir2 for the pixel
        
    tsmask: float, 1D array
        Cloud /shadow mask time series for the pixel
    
    Return:  
    
    Updated cloud/shadow mask time serie
 
    """

    # scale factor
    scale = 10000.0

    # invalid value
    ivd = -999 / scale

    # initialise tsmask, all as clear pixels
    tsmask[:] = 1

    # copy and covert surface reflectance data as float
    blue = blue.copy().astype(np.float32) / scale
    green = green.copy().astype(np.float32) / scale
    red = red.copy().astype(np.float32) / scale
    nir = nir.copy().astype(np.float32) / scale
    swir1 = swir1.copy().astype(np.float32) / scale
    swir2 = swir2.copy().astype(np.float32) / scale

    # detect pixels with invalid data value
    tsmask[blue == ivd] = 0
    tsmask[green == ivd] = 0
    tsmask[red == ivd] = 0
    tsmask[nir == ivd] = 0
    tsmask[swir1 == ivd] = 0
    tsmask[swir2 == ivd] = 0

    # calculate indices

    # mean of 6 spectral bands
    sa = (blue + green + red + nir + swir1 + swir2) / 6

    # modified normalised difference water index
    mndwi = (green - swir1) / (green + swir1)

    # modified soil adjusted vegetation index
    msavi = (2 * nir + 1 - np.sqrt((2 * nir + 1) * (2 * nir + 1) - 8 * (nir - red))) / 2

    # Band different ratio between red and blue
    wbi = (red - blue) / blue

    # Sum of red and blue band
    rgm = red + blue

    # Band different ratio between green and mean of red and blue
    grbm = (green - (red + blue) / 2) / ((red + blue) / 2)

    # Bright cloud theshold
    maxcldthd = 0.45

    # label all ultra-bright pixels as clouds
    tsmask[sa > maxcldthd] = 2

    # detect single cloud / shadow pixels
    testpair(sa, mndwi, 1, tsmask)
    testpair(sa, mndwi, 1, tsmask)
    testpair(sa, mndwi, 1, tsmask)

    # detect 2 consecutive cloud / shadow pixels
    testpair(sa, mndwi, 2, tsmask)
    testpair(sa, mndwi, 2, tsmask)

    # detect 3 consecutive cloud / shadow pixels
    testpair(sa, mndwi, 3, tsmask)

    # detect single cloud / shadow pixels
    testpair(sa, mndwi, 1, tsmask)

    # cloud shadow theshold
    shdthd = 0.05

    # mndwi water pixel theshold
    dwithd = -0.05

    # mndwi baregroud pixel theshold
    landcloudthd = -0.38

    # msavi water pixel theshold
    avithd = 0.06

    # mndwi water pixel theshold
    wtdthd = -0.2

    for i, lab in enumerate(tsmask):

        if lab == 3 and mndwi[i] > dwithd and sa[i] < shdthd:  # water pixel, not shadow
            tsmask[i] = 1

        if lab == 2 and mndwi[i] < landcloudthd:  # bare ground, not cloud
            tsmask[i] = 1

        if (
            lab == 3 and msavi[i] < avithd and mndwi[i] > wtdthd
        ):  # water pixel, not shadow
            tsmask[i] = 1

        if (
            lab == 1
            and wbi[i] < -0.02
            and rgm[i] > 0.06
            and rgm[i] < 0.29
            and mndwi[i] < -0.1
            and grbm[i] < 0.2
        ):  # thin cloud
            tsmask[i] = 2

    return tsmask

In [3]:
def testpair(sa, dwi, N, tsmask):

    """
    Function Name: testpair
    Description: 
    
    This function identifies cloud and shadow pixels in a time series by comparing its value to its neighbours
  
    Parameters:
    
    sa: float, 1D array
        time series of the mean of surface reflectance value of the 6 spectral bands
    dwi: float, 1D array, 
        time series of MNDWI (modified normalised water difference index)
    tsmasak: uint8, 1D array
        time series of cloud/shadow labels
    Return:
    None, tsmask is updated 
    
 
    """
    # cloud detection threshold, the lower the value, the higher cloud detection rate
    cspkthd = 0.42

    # shade detection threshold, the lower the value, the higher shade detection rate
    sspkthd = 0.42

    # the minimum theshold of a cloud pixel, i.e., all cloud pixels will have a band average
    # value higher that this theshold
    cloudthd = 0.10

    # The shadow pixel theshold
    shadowthd = 0.055

    # Find all clear pixels in the time series
    validx = np.where(tsmask == 1)[0]

    # The number of the clear pixels in the time series
    vss = validx.size

    # Not enough clear pixels in the time series
    if vss < 3 * N:
        return

    # Filter out invalid, cloud, shadow points in time series
    vsa = sa[validx]
    vdwi = dwi[validx]
    vtsmask = tsmask[validx]

    # flags which indicates if
    chmarker = np.zeros(vss, dtype=np.int8)

    # flags which indicates a pixel is either a non-shadow or a water pixels
    dws_flags = np.logical_or(vsa > shadowthd, vdwi > 0)

    # Total number of segments in the time series
    numse = vss - N + 1

    # array to store mean of the segments
    msa = np.zeros(numse, dtype=np.float32)

    # calculate mean values of the time series segments

    if N == 1:
        msa = vsa

    else:
        for i in np.arange(numse):
            msa[i] = vsa[i : i + N].sum() / N

    # sort the time series of mean of the segemnts
    sts = np.argsort(msa)

    # reverse the order from ascending to descending, so that sts contains index number of msa array, from
    # highest values to the lowest
    sts = sts[::-1]

    for k in sts:

        if chmarker[k] == 0:
            # mean of the segment
            m2 = msa[k]
            # mean of the neighbouring 2N pixels
            mid = findnbsa(vsa, k, N, vss, dws_flags, vtsmask)

            # check if the mean of segemnt is significantly different from the neighbouring pixels
            if m2 > mid and mid > 0:
                if (m2 - mid) / mid > cspkthd and m2 > cloudthd:
                    # cloud pixels
                    vtsmask[k : k + N] = 2
                    chmarker[k : k + N] = 1

            elif mid > m2 and m2 > 0:
                if (mid - m2) / m2 > sspkthd and m2 < shadowthd:
                    # shadow pixels
                    vtsmask[k : k + N] = 3
                    chmarker[k : k + N] = 1

    # update the orginal time series mask
    tsmask[validx] = vtsmask


In [6]:
def findnbsa(vsa, k, N, vss, dws_flags, vtsmask):

    """
    Function Name: findnbsa
    Description: 
    Given the length of the segment is N, the function will find N clear pixels closest to the segment from the left, N
    N clear pixels closest to the segment from the right, calculate the mean of these 2N pixels
    Parameters:
    
    vsa: float, 1D array
        time series of mean of surface relectance values
    k: integer 
        location of the specified segment
    N: integer
        length of the time series segment
    dws_flags: uint8, 1D array
        flags indicating that a pixel is either a non-shadow pixel or a water pixel
    vtsmask: uint8, 1D array
        time series of cloud/shadow labels
    
    
    Return:  mean values of neighbour pixels of the specified segment
 
    """

    # clear pixel counter
    cc = 0

    # Search direction, 0 -- search the left, 1 -- search the right
    dr = 0

    # Directional flags, 0 -- search can be continued, 1 -- search reach the boundary
    mvd = [0, 0]

    # location of the left of the segment
    lpt = k

    # location of the right of the segment
    rpt = k + N - 1

    # sum of the found clear pixels
    mid = 0.0

    while cc < 2 * N:

        # search the left
        if dr == 0:
            # Not reach the left boundary yet
            if mvd[0] == 0:
                while True:
                    # Move the left pointer to 1 pixel left
                    lpt -= 1
                    # reach the begining of the time series?
                    if lpt < 0:
                        # Yes, modify the directional flags, change the srach directional
                        mvd[dr] = 1
                        dr = 1
                        break
                    elif vtsmask[lpt] == 1 and dws_flags[lpt] == 1:
                        # No, if the pixel is a clear pixels and the pixel is not a potenial shadow pxiels
                        # Add the value of the pixel to the sum of the found clear pixels
                        mid += vsa[lpt]
                        # update the clear pixel counter, change the search direction to right
                        cc += 1
                        dr = 1
                        break
            else:
                dr = 1
        else:
            # search the right
            if mvd[1] == 0:
                # Not reach the right boundary yet
                while True:
                    # Move the right pointer to 1 pixel right
                    rpt += 1
                    # reach the end of the time series?
                    if rpt == vss:
                        # Yes, modify the directional flags, change the srach directional
                        mvd[dr] = 1
                        dr = 0
                        break
                    elif vtsmask[rpt] == 1 and dws_flags[rpt] == 1:
                        # No, if the pixel is a clear pixels and the pixel is not a potenial shadow pxiels
                        # Add the value of the pixel to the sum of the found clear pixels
                        mid += vsa[rpt]
                        # update the clear pixel counter, change the search direction to left
                        cc += 1
                        dr = 0
                        break
            else:
                dr = 0

        # The search reach the boundaries in both direction, exit the search
        if mvd[0] == 1 and mvd[1] == 1:
            break

    # if not enough clear pixels found, return 0, otherwise return the mean of the found 2N clear pixels
    if cc < 2 * N:
        return 0
    else:
        return mid / (2 * N)

In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

from skimage.morphology import dilation
from skimage.morphology import disk
from skimage.morphology import remove_small_objects

i,j=11,3
ds2018 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2018.nc")
ds2019 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2019.nc")

ds = xr.concat([ds2018, ds2019], dim='time').sortby('time')

blue = ds.nbart_blue.values
green = ds.nbart_green.values
red = ds.nbart_red.values
nir = ds.nbart_nir_2.values
swir1 = ds.nbart_swir_2.values
swir2 = ds.nbart_swir_3.values
tsmask = np.zeros((129,400,400))

for j in range(400):
    if j%10 == 0:
        print(j)
    for i in range(400):
        perpixel_filter_direct(blue[:,j,i], green[:,j,i], red[:,j,i], nir[:,j,i], 
                               swir1[:,j,i], swir2[:,j,i], tsmask[:,j,i])

0
10
20
30
40
50
60
70
80
90
100


In [ ]:
ds.nbart_blue.where(tsmask==1).plot(col='time', col_wrap=6, add_colorbar=False)